<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/CNN_GRU_V8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf
from keras.models import Model
from sklearn.metrics import classification_report

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from tensorflow.keras.utils import pad_sequences
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [9]:
# Download the pre-trained GloVe word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-04-14 10:05:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-14 10:05:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-14 10:05:06--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [5]:
data = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]

In [6]:
data.head()

,post_tokens,Target_cat,Label_cat,final_rationales
0,"['i', 'dont', 'think', 'im', 'getting', 'my', ...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"['we', 'cannot', 'continue', 'calling', 'ourse...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"['nawt', 'yall', 'niggers', 'ignoring', 'me']",African,normal,"[0, 0, 0, 0, 0]"
3,"['<user>', 'i', 'am', 'bit', 'confused', 'coz'...",Asian,hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"['this', 'bitch', 'in', 'whataburger', 'eating...",Women,hatespeech,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))


In [ ]:
data = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
for i in range(len(data)):
  
  sentence =""
  for j in (data['post_tokens'].iloc[i]):
    
    sentence += j +" "
    
  data['post_tokens'].iloc[i] = sentence

<ipython-input-6-5f0ee8932d8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
<ipython-input-6-5f0ee8932d8a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'].iloc[i] = sentence


In [10]:
label_map = {'hatespeech': 2, 'normal': 0, 'offensive': 1}
labels = data["Label_cat"].apply(lambda x: label_map[x])

In [11]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y2 = label_encoder.fit_transform(data['Target_cat'])

In [12]:
max_words = 30000
maxlen = 100

# Tokenize the text data and convert it to sequences of integers
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['post_tokens'])
sequences = tokenizer.texts_to_sequences(df['post_tokens'])
X = pad_sequences(sequences, maxlen=maxlen)

# Define the target labels
y1 = pd.get_dummies(df['Label_cat']).values
y2 = to_categorical(y2, num_classes=21)

In [13]:
# Load the pre-trained GloVe embeddings
embedding_dict = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_dict[word] = vector

# Initialize the embedding matrix with pre-trained GloVe embeddings
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, 100))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


## Model Arichtecture

In [14]:
model1 = Sequential()
model1.add(Embedding(num_words, 100, weights=[embedding_matrix], input_length=maxlen, trainable=True))
model1.add(Conv1D(64, 7, activation='relu'))
model1.add(MaxPooling1D(2))
model1.add(Dropout(0.1))
model1.add(GRU(128, recurrent_dropout=0.01))
model1.add(Dense(256, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model2 = Dense(3, activation='softmax')(model1.output)
model3 = Dense(21, activation='softmax')(model1.output)

model = Model(inputs=model1.input, outputs=[model2, model3])
# Compile the model1
model.compile(optimizer=RMSprop(lr=1e-3), loss='categorical_crossentropy', metrics=['acc'])

# Train the model1
history=model.fit(X, [y1, y2], epochs=15, batch_size=32, validation_split=0.3)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/15
441/441 [==============================] - 89s 189ms/step - loss: 2.6899 - dense_3_loss: 0.9024 - dense_4_loss: 1.7875 - dense_3_acc: 0.5798 - dense_4_acc: 0.4948 - val_loss: 3.4256 - val_dense_3_loss: 1.0669 - val_dense_4_loss: 2.3587 - val_dense_3_acc: 0.4243 - val_dense_4_acc: 0.2842
Epoch 2/15
441/441 [==============================] - 81s 182ms/step - loss: 2.1163 - dense_3_loss: 0.7213 - dense_4_loss: 1.3950 - dense_3_acc: 0.6903 - dense_4_acc: 0.5950 - val_loss: 3.7041 - val_dense_3_loss: 1.3030 - val_dense_4_loss: 2.4011 - val_dense_3_acc: 0.4225 - val_dense_4_acc: 0.3196
Epoch 3/15
441/441 [==============================] - 82s 187ms/step - loss: 1.8005 - dense_3_loss: 0.6518 - dense_4_loss: 1.1487 - dense_3_acc: 0.7222 - dense_4_acc: 0.6663 - val_loss: 3.0117 - val_dense_3_loss: 1.0700 - val_dense_4_loss: 1.9417 - val_dense_3_acc: 0.4821 - val_dense_4_acc: 0.4248
Epoch 4/15
441/441 [==============================] - 80s 181ms/step - loss: 1.5812 - dense_3_loss: 0.5

In [15]:
# Get the predicted labels
y_pred1, y_pred2 = model.predict(X)

# Convert the labels from one-hot encoding to integers
y_pred1 = np.argmax(y_pred1, axis=1)
y_pred2 = np.argmax(y_pred2, axis=1)
y_test_int1 = np.argmax(y1, axis=1)
y_test_int2 = np.argmax(y2, axis=1)


630/630 [==============================] - 17s 27ms/step


In [16]:
print("Classification report for model2:")
print(classification_report(y_test_int2,y_pred2))

Classification report for model2:
              precision    recall  f1-score   support

           0       0.81      0.89      0.85      3037
           1       0.76      0.57      0.66       583
           2       0.68      0.68      0.68       326
           3       0.00      0.00      0.00         1
           4       0.24      0.58      0.34       334
           5       0.00      0.00      0.00        33
           6       0.20      0.23      0.21        48
           7       0.00      0.00      0.00        10
           8       0.00      0.00      0.00        15
           9       0.98      0.83      0.90       222
          10       0.93      0.73      0.82      1576
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00         4
          13       0.83      0.76      0.79      1633
          14       0.89      0.83      0.86      1424
          15       0.00      0.00      0.00        89
          16       0.00      0.00      0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
print("Classification report for model3:")
print(classification_report(y_test_int1,y_pred1))

Classification report for model3:
              precision    recall  f1-score   support

           0       0.83      0.84      0.83      5934
           1       0.87      0.85      0.86      7814
           2       0.74      0.75      0.75      6400

    accuracy                           0.82     20148
   macro avg       0.81      0.81      0.81     20148
weighted avg       0.82      0.82      0.82     20148



In [18]:
model.save("/content/drive/MyDrive/Reproducibility/CNN_GRU2.h5")

In [26]:
sentence =  "He is a bloody indian terrorist"
sentence = sentence.lower()
input_seq = tokenizer.texts_to_sequences([sentence])
input_seq = pad_sequences(input_seq, maxlen=maxlen)

# Predict the output
output = model.predict(input_seq)

# Extract the predictions for each task
prediction1 = np.argmax(output[0])
prediction2 = np.argmax(output[1])
label_maps = {2: 'hatespeech', 0: 'normal', 1: 'offensive'}
# Print the predictions
print("Prediction for task 1:", label_maps[prediction1])
print("Prediction for task 2:", label_encoder.inverse_transform([prediction2]))

1/1 [==============================] - 0s 74ms/step
Prediction for task 1: hatespeech
Prediction for task 2: ['Asian']


Note: This examples are taken to just check the working of model. It is not to abuse anyone.